### Import Libraries

In [1]:
# import required libraries
# import kfp
# from kfp import dsl
# from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, HTML, OutputPath, ClassificationMetrics,
#                         Metrics, component)
# from kfp.v2 import compiler
from dateutil.relativedelta import relativedelta

# from google.cloud.aiplatform import pipeline_jobs
# from google_cloud_pipeline_components.v1.batch_predict_job import \
#     ModelBatchPredictOp as batch_prediction_op
from google.cloud import storage
import os
import re
import google
import datetime

from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from dateutil.relativedelta import relativedelta

from pathlib import Path
from datetime import timedelta, date
(datetime.datetime.now()-timedelta(hours=7)).strftime('%Y_%m_%d')


'2023_05_05'

### Parameters

In [2]:
#tag cell with parameters
PROJECT_ID =  'divg-josh-pr-d1cc3a'
BUCKET_NAME='divg-josh-pr-d1cc3a-default'
DATASET_ID = 'whsia_churn_dataset_v2'
RESOURCES_BUCKET = 'divg-josh-pr-d1cc3a-default'
FILE_BUCKET = 'divg-josh-pr-d1cc3a-default'
REGION = 'northamerica-northeast1'

# PROJECT_ID = f"{PROJECT_ID}"
# DATASET_ID = 'whsia_churn_dataset_v2'
# RESOURCE_BUCKET = f"{PROJECT_ID}_bkt_whsia_churn_v2"
# FILE_BUCKET = f"{PROJECT_ID}_bkt_whsia_churn_v2"

### Service Parameters

In [3]:
SERVICE_TYPE = 'whsia-churn-v2'
TABLE_ID = 'bq_whsia_churn_score_v2'
REGION = "northamerica-northeast1"

### Pulumi Parameters

In [4]:
STACK_NAME = 'whsia_churn_v2'
SERVING_PIPELINE_NAME_PATH = 'whsia_churn_v2_model/serving-pipeline'
PIPELINE_NAME = 'whsia-churn-v2-serving-pipeline' # Same name as pulumi.yaml
PIPELINE_DESCRIPTION = 'whsia-churn-v2-serving-pipeline'
REGION = "northamerica-northeast1"

### Query + Pre-Processing Component Parameters

In [5]:
QUERIES_PATH = f"{STACK_NAME}/queries/" #check
UTILS_FILE_PATH = f"{STACK_NAME}/{SERVING_PIPELINE_NAME_PATH}/utils" # check
UTILS_FILENAME = 'utils.py'

PROCESSED_SERVING_DATA_TABLENAME = 'processed_serving_data'
INPUT_SERVING_DATA_TABLENAME = 'input_serving_data'

QUERY_DATE = (date.today() - relativedelta(days=1)).strftime('%Y-%m-%d')
wHSIA_QUERY_VIEW_NAME = 'whsia_query_path_view'
wHSIA_QUERY_PATH = QUERIES_PATH + 'whsia_test_qry_v5.txt'
TARGET_TABLE_REF = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TABLE_ID)

### Import Pipeline Components

In [6]:
# # download required component files to local
# prefix = f'{STACK_NAME}/{SERVING_PIPELINE_NAME_PATH}/components/'
# dl_dir = 'components/'

# storage_client = storage.Client()
# bucket = storage_client.bucket(RESOURCES_BUCKET)
# blobs = bucket.list_blobs(prefix=prefix)  # Get list of files
# for blob in blobs: # download each file that starts with "prefix" into "dl_dir"
#     if blob.name.endswith("/"):
#         continue
#     file_split = blob.name.split(prefix)
#     file_path = f"{dl_dir}{file_split[-1]}"
#     directory = "/".join(file_path.split("/")[0:-1])
#     Path(directory).mkdir(parents=True, exist_ok=True)
#     blob.download_to_filename(file_path) 

# # import main pipeline components
from components.create_wHSIA_view import create_wHSIA_view
from components.wHSIA_processing import wHSIA_processing

# import components

### Pipeline

In [7]:

def pipeline(
        project_id: str = PROJECT_ID,
        region: str = REGION,
        resource_bucket: str = RESOURCES_BUCKET
):

    # -------------  create ops ---------------
    create_wHSIA_view_op = create_wHSIA_view(
        view_name=wHSIA_QUERY_VIEW_NAME,
        query_date=QUERY_DATE,
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        region=REGION,
        resource_bucket=RESOURCES_BUCKET,
        query_path=wHSIA_QUERY_PATH
    )        

    wHSIA_processing_op = wHSIA_processing(
        view_name=wHSIA_QUERY_VIEW_NAME,
        project_id=PROJECT_ID,
        dataset_id=DATASET_ID,
        table_id=TABLE_ID,
        query_date=QUERY_DATE,
        file_bucket=FILE_BUCKET
    )
    
    create_wHSIA_view_op
    wHSIA_processing_op
    
    
    
    

In [8]:
pipeline(project_id = PROJECT_ID, region = REGION, resource_bucket = RESOURCES_BUCKET)

......wHSIA table generated with 10000 samples
...... will upsert 6 batches
(10000, 4)
0 batch upserted
(2000, 4)
         ban  score_date model_id     score
0  223486692  2023-03-31     5070  0.187654
1  224096547  2023-03-31     5070  0.188406
2  223574390  2023-03-31     5070  0.192973
3  224038121  2023-03-31     5070  0.189203
4  223559295  2023-03-31     5070  0.188361
5  223870384  2023-03-31     5070  0.192520
6  224098889  2023-03-31     5070  0.188410
7  223534219  2023-03-31     5070  0.187599
8  223874236  2023-03-31     5070  0.189648
9  224165398  2023-03-31     5070  0.192842
predict for batch 0 done 1 batch upserted
(2000, 4)
            ban  score_date model_id     score
2000  600214073  2023-03-31     5070  0.189777
2001  600884473  2023-03-31     5070  0.199707
2002  600771631  2023-03-31     5070  0.192397
2003  600590787  2023-03-31     5070  0.190089
2004  600799465  2023-03-31     5070  0.188040
2005  600426533  2023-03-31     5070  0.187996
2006  600204825  2023

### Run the Pipeline Job

In [9]:
# from kfp.v2 import compiler
# from google.cloud.aiplatform import pipeline_jobs

# import json

# compiler.Compiler().compile(
#    pipeline_func=pipeline, package_path="pipeline.json"
# )

# job = pipeline_jobs.PipelineJob(
#                                display_name=PIPELINE_NAME,
#                                template_path="pipeline.json",
#                                location=REGION,
#                                enable_caching=False,
#                                pipeline_root = f"gs://{RESOURCES_BUCKET}"
# )
# job.run(
#    service_account = f"bilayer-sa@{PROJECT_ID}.iam.gserviceaccount.com"
# )